In [9]:
import networkx as nx
import matplotlib.pyplot as plt
from neo4j import GraphDatabase
from dotenv import load_dotenv
from pyvis.network import Network
from neo4j import GraphDatabase
import os

In [ ]:
# Get the URI and authentication credentials from environment variables
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

# initalize the driver
driver = GraphDatabase.driver(URI, auth=AUTH)

In [11]:
# Query the graph to extract nodes and relationships
def fetch_graph_data():
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    nodes = {}
    relationships = []
    with driver.session() as session:
        for record in session.run(query):
            # Add nodes
            for node in [record["n"], record["m"]]:
                node_id = node["name"]
                if node_id not in nodes:
                    nodes[node_id] = {
                        "id": node_id,
                        "type": list(node.labels)[0],  # Node type (Variable, Source, Endpoint)
                        "properties": dict(node)      # Node properties (e.g., format, label)
                    }

            # Add relationships
            relationships.append({
                "type": record["r"].type,
                "startNode": record["r"].start_node["name"],
                "endNode": record["r"].end_node["name"]
            })

    return nodes, relationships

# Visualize the graph with Pyvis
def visualize_knowledge_graph_interactive():
    nodes, relationships = fetch_graph_data()

    # Create a Pyvis network
    net = Network(height="750px", width="100%", directed=True)

    # Add nodes
    for node_id, node_data in nodes.items():
        net.add_node(
            node_id,
            label=f"{node_data['type']} ({node_id})",  # Label shows type and node ID
            title=str(node_data["properties"]),       # Tooltip shows properties
            shape="dot"                               # Use a dot shape for simplicity
        )

    # Add edges
    for rel in relationships:
        net.add_edge(
            rel["startNode"],
            rel["endNode"],
            title=rel["type"],  # Tooltip shows relationship type
            label=rel["type"]   # Edge label shows the relationship type
        )

    # Generate and open the graph
    net.show("knowledge_graph.html")

# Run the visualization
visualize_knowledge_graph_interactive()

knowledge_graph.html


AttributeError: 'NoneType' object has no attribute 'render'